In [8]:
# Vorbereitung der Daten und Erstellen von zwei neuen CSV-Dateien

In [ ]:
import pandas as pd
data = pd.read_csv("ladesaeulenregister.csv", delimiter=";", on_bad_lines="skip")


C:\Users\petra\AppData\Local\Temp\ipykernel_16056\863527573.py:2: DtypeWarning: Columns (21,24) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("ladesaeulenregister.csv", delimiter=";", on_bad_lines="skip")


In [10]:
data.columns

Index(['Betreiber', 'Straße', 'Hausnummer', 'Adresszusatz', 'Postleitzahl',
       'Ort', 'Bundesland', 'Kreis/kreisfreie Stadt', 'Breitengrad',
       'Längengrad', 'Inbetriebnahmedatum',
       'Nennleistung Ladeeinrichtung [kW]', 'Art der Ladeeinrichung',
       'Anzahl Ladepunkte', 'Steckertypen1', 'P1 [kW]', 'Public Key1',
       'Steckertypen2', 'P2 [kW]', 'Public Key2', 'Steckertypen3', 'P3 [kW]',
       'Public Key3', 'Steckertypen4', 'P4 [kW]', 'Public Key4'],
      dtype='object')

In [11]:
# Unnötige Spalten / Zeilen entfernen
data = data.drop(columns=['Adresszusatz', 'Public Key1', 'Public Key2', 'Public Key3', 'Public Key4' ])
data = data.dropna(subset=['Inbetriebnahmedatum'])
data = data.dropna(subset=['Betreiber'])

# Jahr der Inbetriebnahme in neue Spalte
data['Inbetriebnahmejahr'] = pd.to_datetime(data['Inbetriebnahmedatum'], format='%d.%m.%Y', errors='coerce').dt.year

# Datentyp Float für numerische Spaltenwerte:
columns_to_convert = ["P1 [kW]", "P2 [kW]", "P3 [kW]", "Nennleistung Ladeeinrichtung [kW]" ]
for col in columns_to_convert:
    data[col] = data[col].replace(',', '.', regex=True)
    data[col] = pd.to_numeric(data[col], errors='coerce')

# Neue Spalte für Ladegeschwindigkeit
def ladegeschwindigkeit(row):
    if row['P1 [kW]'] <= 22:
        return 'Normalladen'
    elif 22 < row['P1 [kW]'] <= 150:
        return 'Schnellladen'
    else:
        return 'High Power Charging'

# Funktion auf Zeilen anwenden und neue Spalte hinzufügen
data['Ladegeschwindigkeit'] = data.apply(ladegeschwindigkeit, axis=1)

data['Breitengrad'] = data['Breitengrad'].str.replace(',', '.').str.rstrip('.').astype(float)
data['Längengrad'] = data['Längengrad'].str.replace(',', '.').str.rstrip('.').astype(float)

In [12]:
data.to_csv("ladesaeulen.csv", index=False, sep=";")


In [13]:
# Umwandlung der Daten: Jede Ladepunkt-Information in eine eigene Zeile
rows = []
for _, row in data.iterrows():
    for i in range(1, 5):  # Für Steckertypen1 bis Steckertypen4
        steckertyp = row[f"Steckertypen{i}"]
        p_kw = row[f"P{i} [kW]"]
        
        if pd.notna(steckertyp):  # Nur Einträge mit Steckertypen behalten
            new_row = row.drop([f"Steckertypen{j}" for j in range(1, 5)] +
                               [f"P{j} [kW]" for j in range(1, 5)]).to_dict()
            new_row.update({
                "Steckertyp": steckertyp,
                "Leistung [kW]": p_kw,
            })
            rows.append(new_row)

# Neues DataFrame erstellen
data_ladeplaetze = pd.DataFrame(rows)

In [14]:
data_ladeplaetze.to_csv("ladeplaetze.csv", index=False, sep=";")